In [ ]:
!pip install pytrends

In [ ]:
!pip install seaborn

In [1]:
from pytrends.request import TrendReq
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import json
import time

In [2]:
timezone_offset = -330 # INDIA
# timezone_offset = -480 # Phillipines

pytrends = TrendReq(hl='en-US', tz=timezone_offset, retries=10, backoff_factor=0.5)

In [3]:
date_entry = input('Enter a start date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

Enter a start date in YYYY-MM-DD format: 2019-01-01


In [4]:
date_entry = input('Enter a end date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date2 = datetime.date(year, month, day)

Enter a end date in YYYY-MM-DD format: 2020-02-02


In [5]:
t_frame = date1.strftime("%Y-%m-%d") + " " + date2.strftime("%Y-%m-%d")
t_frame

'2019-01-01 2020-02-02'

In [6]:
cat_df = pd.read_csv("categories_defined.csv")
cat_names = cat_df["name"].tolist()
cat_ids = cat_df["id"].tolist()

In [14]:
def get_related_queries(catno, timeframe):
    """
    Gets related queries
    """
    kw_list = [""]
    # Create Pytrend Client
    pytrends.build_payload(cat=catno, kw_list=kw_list, timeframe=timeframe, geo="PH")
    
    # Form Request
    related_payload = dict()
    request_json = pytrends.related_queries_widget_list[0]
    related_payload["req"] = json.dumps(request_json["request"])
    related_payload["token"] = request_json["token"]
    related_payload["tz"] = pytrends.tz
    
    # Send Request
    req_json = pytrends._get_data(
        url=TrendReq.RELATED_QUERIES_URL,
        method=TrendReq.GET_METHOD,
        trim_chars=5,
        params=related_payload,
    )
    
    # return req_json
    # Tabulate Rising & Top searches.
    try:
        top_df = pd.DataFrame(req_json["default"]["rankedList"][0]["rankedKeyword"])
        top_df = top_df[["query", "formattedValue"]]
    except KeyError:
        # in case no top queries are found, the lines above will throw a KeyError
        top_df = pd.DataFrame(columns=["query", "formattedValue"])

    # rising queries
    try:
        rising_df = pd.DataFrame(req_json['default']['rankedList'][1]["rankedKeyword"])
        rising_df = rising_df[['query', 'formattedValue']]
    except KeyError:
        # in case no rising queries are found, the lines above will throw a KeyError
        rising_df = pd.DataFrame(columns=["query", "formattedValue"])

    return(top_df,rising_df)

In [22]:
related_queries = pd.DataFrame()

for i in range(len(cat_ids)):
    # print(i, cat_ids[i], cat_names[i])
    if(i%20==0):
        time.sleep(20)
    rq_top_df, rq_rising_df = get_related_queries(cat_ids[i], t_frame)
    if(len(rq_top_df)==0):
        print(cat_names[i], "has no top queries")
    if(len(rq_rising_df)==0):
        print(cat_names[i], "has no rising queries")
    
    rq_top_df.columns = ["value", "subject"]
    rq_rising_df.columns = ["value", "subject"]
    
    rq_top_df["related_queries"] = "top"
    rq_rising_df["related_queries"] = "rising"
    
    rq_df = rq_top_df.append(rq_rising_df)
    
    rq_df["geo"] = "PH"
    rq_df["keyword"] = cat_names[i]
    rq_df["category"] = cat_ids[i]
    related_queries = related_queries.append(rq_df)

Babies & Toddlers has no top queries
Babies & Toddlers has no rising queries
Baby Care & Hygiene has no top queries
Baby Care & Hygiene has no rising queries
Bedding & Bed Linens has no top queries
Bedding & Bed Linens has no rising queries
Beds & Headboards has no top queries
Beds & Headboards has no rising queries
Camera Lenses has no top queries
Camera Lenses has no rising queries
Cameras has no top queries
Cameras has no rising queries
Cameras & Camcorders has no top queries
Cameras & Camcorders has no rising queries
Car Audio has no top queries
Car Audio has no rising queries


ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=2)

In [21]:
related_queries.reset_index().to_csv("related_queries.csv")